# AI Skillet 
## Sandeep Pawar | 08-30-2024
### Requirements:

- Gradio
- plotly
- powerbiclient
- AI Skills published with an endpoint

To use inline in the notebook `demo.launch(inline=True)`

Limitations:
- One workspace at a time
- Manually enter AI Skills GUIDs. You can enter as many AI skills as you want.
- Plotting is basic.
- Schema not validated.
- Results can be incorrect and not validated. 

Steps:

- Update workspace id and ai skills ids
- install dependencies
- Execute cells
- Open the app in a browser (http://127.0.0.1:7860)
- Click on AUthenticate to authenticate
- Ask questions

### Install Dependencies & Import Libraries

In [1]:
!pip install gradio pydantic powerbiclient plotly --q

In [2]:
import gradio as gr
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from powerbiclient.authentication import InteractiveLoginAuthentication
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The value passed into gr.Dropdown()")
warnings.filterwarnings("ignore", category=UserWarning, message="The `value` received is not in `choices`.")

### Add AI Skills

In [3]:
TOKEN = None

#Enter youe workspace ID
WORKSPACE_ID = "cdxxxx9d-xxx-xxxxxxxxxxxx94"

#Add AI Skills IDs and names. There is no API yet to get the list of AI Skills pubslihe so you have to do this manually.

AI_SKILLS = {
    "Sales Analysis": "xxxxxxxxxxxxxxxx",
    "Customer Growth": "xxxxxxxxxxxxxxxxxxx",
    "Global Dispatch Forecast": "xxxxxxxxxxxxxxxxxxxxx"
}

### Utilities

In [4]:
## interactive authentication
def authenticate():
    global TOKEN
    interactive_auth = InteractiveLoginAuthentication()
    TOKEN = interactive_auth.get_access_token()
    return "Authenticated" if TOKEN else "Authentication failed"

# Call the API 
def chat(message, history, ai_skill):
    global TOKEN
    if not TOKEN:
        return "", history + [("You", message), ("AI Skillet | Fabric.Guru", "Please authenticate first.")], None, None, None
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {TOKEN}'
    }
    
    ai_skill_id = AI_SKILLS[ai_skill]
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_ID}/aiskills/{ai_skill_id}/query/deployment"
    data = json.dumps({"userQuestion": message})
    
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        result = response.json()
        
        result_header = result['ResultHeaders']
        result_rows = result['ResultRows']
        
        df = pd.DataFrame(result_rows, columns=result_header)
        
        ai_response = f"Here's the result for your query using {ai_skill}:"
        return "", history + [("You", message), ("AI Skill", ai_response)], df, df.columns.tolist(), df.columns.tolist()
    except requests.RequestException as e:
        error_msg = f"Error: {str(e)}"
        return "", history + [("You", message), ("AI Skill", error_msg)], None, None, None

#Create plotly visualizations
def create_plot(df, plot_type, x_column, y_columns):
    if df is None or df.empty or not x_column or not y_columns:
        return gr.Plot(visible=False)
    
    try:
        if isinstance(y_columns, list) and len(y_columns) == 1:
            y_columns = y_columns[0]  # If only one y-column is selected, convert it to a string
        
        if plot_type == "Line Plot":
            fig = px.line(df, x=x_column, y=y_columns)
        elif plot_type == "Bar Plot":
            fig = px.bar(df, x=x_column, y=y_columns)
        elif plot_type == "Scatter Plot":
            fig = px.scatter(df, x=x_column, y=y_columns)
        else:
            fig = px.line(df, x=x_column, y=y_columns)  # Default to line plot
        
        return gr.Plot(value=fig, visible=True)
    except Exception as e:
        print(f"Error creating plot: {str(e)}")
        return gr.Plot(visible=False)

def update_plot(df, plot_type, x_column, y_columns):
    if df is not None and not df.empty and x_column and y_columns:
        return create_plot(df, plot_type, x_column, y_columns)
    return gr.Plot(visible=False)

# Chat interface using Gradio

def create_interface():
    with gr.Blocks(css="""
                #app-title { 
            color: #1c4e4e;
            font-size: 32px;
            font-weight: 900;
            margin-bottom: 0;
            text-transform: uppercase;
            letter-spacing: 1px;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.1);
            font-family: 'Arial Black', Helvetica, sans-serif;
            padding: 10px 0;
            border-bottom: 2px solid #1c4e4e;
        }
        #app-subtitle { color: #1c4e4e; font-size: 18px; margin-top: 5px; }
        .gradio-container { background-color: #faf7f0; }
        .gr-button { background-color: #1c4e4e; color: white; }
        .gr-button:hover { background-color: #163d3d; }
        .gr-dropdown { background-color: #e8e6dd; }
        .gr-input { background-color: #e8e6dd; }
    """) as demo:
        gr.Markdown("<p id='app-title'>AI Skillet</p>")
        gr.Markdown("<p id='app-subtitle'>Local AI Skills App | Fabric.Guru | v0.2</p>")
        
        with gr.Row():
            with gr.Column(scale=2):
                ai_skill_dropdown = gr.Dropdown(choices=list(AI_SKILLS.keys()), label="Select AI Skills", value="Sales Analysis")
            with gr.Column(scale=1):
                auth_button = gr.Button("Authenticate")
                auth_output = gr.Textbox(label="Authentication Status", interactive=False)
        
        chatbot = gr.Chatbot(height=300)
        msg = gr.Textbox(placeholder="Type your message here...")
        result_df = gr.Dataframe(label="Query Result", interactive=False)
        
        with gr.Row():
            plot_type = gr.Dropdown(choices=["Line Plot", "Bar Plot", "Scatter Plot"], label="Select Plot Type", value="Line Plot")
            x_column = gr.Dropdown(label="Select X-axis Column", interactive=False)
            y_columns = gr.Dropdown(label="Select Y-axis Column(s)", multiselect=True, interactive=False)
        
        plot_output = gr.Plot(label="Visualization", visible=False)
        
        clear = gr.Button("Clear")
        
        def update_column_options(df):
            if df is not None and not df.empty:
                columns = df.columns.tolist()
                return (
                    gr.Dropdown(choices=columns, value=None, interactive=True),
                    gr.Dropdown(choices=columns, value=None, interactive=True, multiselect=True)
                )
            return (
                gr.Dropdown(choices=[], value=None, interactive=False),
                gr.Dropdown(choices=[], value=None, interactive=False, multiselect=True)
            )
        
        msg.submit(chat, inputs=[msg, chatbot, ai_skill_dropdown], 
                   outputs=[msg, chatbot, result_df, x_column, y_columns])
        
        result_df.change(update_column_options, inputs=[result_df], outputs=[x_column, y_columns])
        
        plot_type.change(update_plot, inputs=[result_df, plot_type, x_column, y_columns], outputs=[plot_output])
        x_column.change(update_plot, inputs=[result_df, plot_type, x_column, y_columns], outputs=[plot_output])
        y_columns.change(update_plot, inputs=[result_df, plot_type, x_column, y_columns], outputs=[plot_output])
        
        def clear_interface():
            return (
                None,
                None,
                gr.Plot(visible=False),
                gr.Dropdown(choices=[], value=None, interactive=False),
                gr.Dropdown(choices=[], value=None, interactive=False, multiselect=True)
            )
        
        clear.click(clear_interface, outputs=[chatbot, result_df, plot_output, x_column, y_columns])
        auth_button.click(authenticate, outputs=auth_output)
    
    return demo



### Launch the App

If you want to use it in the notebook, set `inline=True`

In [5]:
demo = create_interface()
demo.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7881

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


A local browser window will be open for interactive sign in.
You have logged in.
Interactive authentication successfully completed.
